In [ ]:
#TPOT is built on top of scikilearn that uses genetic programming to optimize our machine learning pipeline.
#TPOT automates these steps(feature selection, feature preprocessing, feature construction, model selection and parameter optimization) for us with genetic programming
# and it will output the optimal code for us when it's done.
# ===========  MUST ACTIVATE TENSORFLOW TO USE TPOT!!!!!!!!! ================
from tpot import TPOTClassifier
#Ml library
from sklearn.cross_validation import train_test_split
#help analyse our data
import pandas as pd 
#perform math calculation
import numpy as np
import sys

#load the data
telescope=pd.read_csv('MAGIC Gamma Telescope Data.csv')
#clean the data
#since the class object is already organized, we'll shuffle our data to get a better result
# the iloc() function of the telescope variable is pandas's way of getting the positions in the index.
# and we'll generate a sequence of random indices the size of our data using the permutation function of numpy's 'random' submodule.
telescope_shuffle=telescope.iloc[np.random.permutation(len(telescope))]
# since all the instances are now randomly rearranged, we'll just reset all these indices so thet are ordered ven though the data is now shuffled using the reset_index()
tele=telescope_shuffle.reset_index(drop=True)

#Store 2 classes
# now let the tele variable know what our two classes are by mapping both of them to an integer with the map()
# Change label 'g' 'h' to numeric '0' '1'
tele['Class']=tele['Class'].map({'g':0, 'h':1})
# store those class labels which we're going to predict
# using .values to retrieve them
tele_class = tele['Class'].values

#Split training, testing, and validation data
#train_test_split() to create indices for both
# we want our dataset to be arrays so we set the stratify= tele_class
# define what percentage of our data to be training anf testing
training_indices, validation_indices = training_indices, testing_indices = train_test_split(tele.index,
    stratify= tele_class, train_size=0.75, test_size=0.25)

#print ("testing!!!")
#sys.exit()

#Let Genetic Programming find best ML model and hyperparameters
#initiate tpot
#generations=5 bc on a standard laptop with 4 gigs of RAM, it takes five minutes per generation to run,so this will take about 25mins
# this is so TPOT's genetic algorithm knows how many iterations to run for
# set verbosity=2 which means "show a progress bar in terminals during the optimazation process"
tpot = TPOTClassifier(generations=5, verbosity=2)
# call our fit() method on our training data to let it perform optimization using genetic programming.
# the first para i the training feature set which we retrieve from tele variable along the first access for every training index
# the second varibale is our training class set which we'll tertieve from our 'tele' variable like so
tpot.fit(tele.drop('Class', axis=1).loc[training_indices].values,
    tele.loc[training_indices, 'Class'].values)

#Score the accuracy
#compute the testing ettor for validation using TPOT's score() method with validation feature set as first para and the valid class set as the second
tpot.score(tele.drop('Class', axis=1).loc[validation_indices].values,
    tele.loc[validation_indices, 'Class'].values)

#Export the generated code
#export the computed python code to the pipeline.py class using this method and name it in the parameter as a string
tpot.export('pipeline.py')
